#Extraction des données du lexique libanais de Makki

Ce document permet de transformer vos données (format docx) en un format exportable sur la plateforme du Lexique libanais de Makki en tant qu'administrateur (format rdf).

## Installation des extensions

In [ ]:
!pip install python-docx==1.1.0
!pip install isodate==0.6.1
!pip install lxml==5.1.0
!pip install pillow
!pip install pyparsing==3.1.1
!pip install rdflib==7.0.0
!pip install six
!pip install typing-extensions==4.9.0

In [ ]:
#Pour docx vers json
import json
from docx import Document
import re

#Pour json vers rdf
import os

#Pour rdf_id
import uuid
import hashlib

#Pour main
import argparse
import time
import sys

##Transformation d'un fichier docx en un fichier json

In [ ]:
def data_to_uuid(*data: tuple[str])->uuid.UUID:
    s = hashlib.md5(bytes(".".join(list(data)), 'utf-8'))
    return uuid.UUID(bytes=s.digest())

In [ ]:
class ExtractData:

    prefixes_fr = {'[Imper.]', '[Jeux]', '[Alim.]', '[Bot.]', '[Zool.]', '[Zool. ; ois.]', '[Zool./Mar. ; poiss.]', '[Zool. ; insec.]', '[Méd./Mal.]', '[Mus.]', '[Mus. ; instr.]', '[Loc. ; vulg.]', '[Pop. ; vulg.]', '[Vulg.]', '[Compar. ; vulg.]'}
    prefixes_ar = {'[لِلأمر]', '[ألعاب]', '[غِذاء]', '[نَبات]', '[حَيَوان]', '[حَيَوان؛ طُيُور]', '[حَيَوانٌ بَـحرِيّ؛ أسـماك]', '[حَيَوان؛ حَشَرات]', '[طِبّ]', '[مُوسِيقى]', '[مُوسِيقى؛ آلَة]', '[عِبارَة؛ سُوقِيّ]', '[شَعبِيّ؛ سُوقِيّ]', '[سُوقِيّ]', '[لِلـمُقارَنَة؛ سُوقِيّ]'}
    themes_ok_fr={'Jeux', 'Zoologie','Médecine','Botanique', 'Technique','Alimentation'}
    themes_ok_ar={'ألعاب','حَيَوانٌ', 'طِبّ','نَبات','تِقَنِيّات','غِذاء'}

    cols_lang = ["ar", "ar", "fr", "ar"]

    regex_coverage = re.compile(r"(\((?P<def>[\d])\) |)((?P<first_loc>[\w ]+), |)(((?P<loc1>[\w]+)( (\((?P<prec1>.+)\))|) (&|et) (?P<loc2>[\w]+)( \((?P<prec2>.+)\)|))|(?P<loc_glob>[\w ]+) (\((?P<loc>[\w ]+)\))|(?P<loca>[\w ]+[\w]+)( (\((?P<prec3>.+)\))|))( : (?P<precision>.+[^\n ]{1})|)", re.M | re.U)
    regex_definition = re.compile(r"^(?P<head>(?P<nb>[0-9]\.) |(\((?P<symbol>.{1})\) )| ?® ?|)((?P<tag>\[.+\]) |)(- (?P<example>.+?)|(?P<def>.+?)) *$", re.M | re.U)
    #regex_definition = re.compile(r"^(?P<head>(?P<nb>[0-9]\.) |(\((?P<symbol>.*)\) )| ?® ?|)((?P<tag>\[.+\]) |)(- (?P<example>.+?)|(?P<def>.+?)) *$", re.M | re.U)






    def __init__(self, file_path, annotations_file_path):
        self.file_path = file_path
        self.annotations_file_path = annotations_file_path
        self.data = {}





    def generate_term_uri(self, description_id):
        return str(data_to_uuid(description_id))





    def make_term(self, terme, description_fr, description_ar):
        data = {}

        # On rassemble les informations des descriptions en français et en arabe
        for key in description_fr.keys() | description_ar.keys():

            if key in description_fr and key in description_ar:
                data[key] = description_fr[key] + description_ar[key]
            elif key in description_fr:
                data[key] = description_fr[key]
            else:
                data[key] = description_ar[key]

        data["title"] = terme

        return data




    def parse_term(self, s: str):
        # On découpe la case du mot en séparant avec les tirets
        s = [el.strip() for el in s.split('-')]

        # On isole le mot lui-même de ses exemples
        #NOTE: on perd actuellement les informations de pluriel, de féminin, etc.
        return re.match(r"[^\d\(\)\n\r\f]+", s[0]).group(0).strip(), [(ex, "ar") for ex in s[1:]]




    def parse_coverage(self, s: str, lang: str):
        output = []
        s = s.lower()
        # On utilise la regex `regex_coverage` pour découper la chaîne en différentes informations
        for m in self.regex_coverage.finditer(s.strip()):

            prec = m['precision']

            if m['first_loc']:
                output.append((m['first_loc'], lang, prec))

            elif m['loc1']:
                output.append((m['loc1'].lower(), lang, m['prec1'] if m['prec1'] else prec))
                output.append((m['loc2'].lower(), lang, m['prec2'] if m['prec2'] else prec))

            elif m['loc_glob']:
                output.append((m['loc_glob'].lower(), lang, prec))
                output.append((m['loc'].lower(), lang, prec))

            elif m['loca']:
                output.append((m['loca'].lower(), lang, m['prec3'] if m['prec3'] else prec))

        return output




    def parse_definition(self, s: str, tags: set, lang: str):
        output = {"abstract" : [""]}
        output['quic'] = False

        valid_themes = self.themes_ok_fr if lang == "fr" else self.themes_ok_ar

        # On utilise la regex `regex_definition` pour découper la définition en éléments reconnus
        for d in self.regex_definition.finditer(s):

            d = d.groupdict()
            #print(f"Détection : {d}")



            # On traite chaque cas possible
            if d['symbol'] == '*':
                output['abstract'][0] += '\n(*)' + d['def'].strip()

            elif d['symbol'] == '+':
                if 'related' not in output:
                    output['related'] = []
                output['related'].append((d['def'].strip(), lang))


            elif d['symbol'] == 'ε':

                if 'etymo' not in output:
                    output['etymo'] = []

                output['etymo'].append((d['def'].strip(), lang))

            elif d['symbol'] == 'π':
                if 'pron' not in output:
                    output['pron'] = []
                output['pron'].append((d['def'].strip(), lang))

            elif d['head'].strip() == "®":
                if lang == "fr":
                    if 'coverage' not in output:
                        output['coverage'] = []
                    output['coverage'] += self.parse_coverage(d['def'], lang)
                else:
                    output['abstract'][0] += '\n' + d['def'].strip()


            elif not d['symbol']:
                if d['def']:

                    if d['tag'] and d['tag'] != '':
                      #print(f"Tag extrait : {d['tag']}")
                      #tg = d['tag'].split('/')[0].split(';')[0].strip().rstrip('[]')
                      tg = d['tag'].split('/')[0].split(';')[0].strip().strip('[]')
                      #print(f"Tag modifié : {tg}")  # Debug


                      #print(f"Tag extrait : {tg}, Thèmes valides : {valid_themes}")

                      # Ne traiter que les tags correspondant aux thèmes valides
                      if tg in valid_themes:
                          if tg not in tags:
                              if 'subject' not in output:
                                  output['subject'] = []
                              output['subject'].append((tg[0:].strip(), lang))
                          else:
                              output['abstract'][0] += d['tag'] + ' '

                    output['abstract'][0] += d['def']

                elif d['example']:
                    if 'example' not in output:
                            output['example'] = []

                    output['example'].append((d['example'].strip(), lang))

        if d['def'] is not None and "¤" in d['def']:
          output['quic'] = True
        output['abstract'][0] = (output['abstract'][0], lang)

        return output





    def parse_row(self, row, tags):
        #print([cell.text for cell in row.cells])

        if len(row.cells) > 2 and row.cells[0].text.strip() != "" and row.cells[2].text.strip() != "":

            # On itère sur les lignes qui ont un mot de présent dans la première colonne
            if row.cells[0].text.strip() != "" and row.cells[2].text.strip() != "":

                # On récupère le terme et les examples
                terme, examples = self.parse_term(row.cells[0].text)

                # On récupère chaque description et les tags valides associés (= pas dans annotations.txt)
                description_fr = self.parse_definition(row.cells[2].text, tags, "fr")
                description_ar = self.parse_definition(row.cells[1].text, tags, "ar")

                # On ajoute le terme au dictionnaire ainsi que les définitions associées
                term_uri = self.generate_term_uri(description_fr['abstract'][0][0])

                if not terme in self.data:
                    self.data[terme] = {"description" : {}}

                self.data[terme]["description"][term_uri] = self.make_term(terme, description_fr, description_ar)


                # On ajoute les exemples
                if len(examples) > 0:
                    if not 'example' in self.data[terme]["description"][term_uri]:
                        self.data[terme]["description"][term_uri]['example'] = []

                    self.data[terme]["description"][term_uri]["example"] += examples

                # On ajoute les synonymes en leur donnant la même définition que le mot actuel
                if (alternatif := re.match(r"(\([0-9]\)|) ?← (.*)", row.cells[3].text)) != None:

                    for alt in alternatif.group(2).split("،"):
                        alt = alt.strip()
                        if alt != "":
                            if not alt in self.data:
                                self.data[alt] = {"description" : {}}

                            self.data[alt]["description"][term_uri] = self.make_term(alt, description_fr, description_ar)




    def extract_definition(self, verbose=True):
        # On récupère le fichier de données et l'annexe des tags à éliminer
        document = Document(self.file_path)
        tags, symbols = self.read_annotations_file()
        # TODO: prendre en compte les symboles
        # On itère sur les tableaux du document
        for i, table in enumerate(document.tables):
            # Si l'on a suffisamment de colonnes, on itère sur les lignes
            if len(table.columns) >= 4:
                for i, row in enumerate(table.rows):
                    if verbose:
                        print(f"\t\t| {i+1:05} / {len(table.rows):05} |")
                    self.parse_row(row, tags)



    def read_annotations_file(self):
        tags = []
        symbols = []

        try:
            with open(self.annotations_file_path, mode="r", encoding="utf-8") as file:
                annotations_to_delete = file.readlines()

                for annotation in annotations_to_delete:
                    if not (annotation[0] in {'[', '('}):
                        symbols.append(annotation.strip())
                    else:
                        tags.append(annotation.strip())
        except:
            print("Fichier d'exclusion INVALIDE :", self.annotations_file_path)
            pass

        return (set(tags), set(symbols))

    def write_json_to_file(self, file_path):
        with open(file_path, mode="w", encoding="utf-8") as file:
            json.dump(self.data, file, ensure_ascii=False, indent=4)

##Transformation d'un fichier json en un fichier rdf

In [ ]:
def convert_to_rdf(input_json, source_file_name):
    rdf_prefixes = """    @prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
    @prefix rdfs:  <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix xsd:   <http://www.w3.org/2001/XMLSchema#> .
    @prefix dc:    <http://purl.org/dc/terms/> .
    @prefix lex:   <lexique/> .

    """

    rdf_template = """<lexique/{title_uuid}#{identifier}>
        dc:creator "{creator}" ;
        dc:publisher "{publisher}" ;
        dc:identifier "{identifier}" ;
        dc:source "{source}" ;
        dc:title "{title}"@ar ;
        {coverage}
        {example}
        {etymo}
        {subject}
        {quic}
        {abstract} .
        """

    with open(input_json, mode='r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    source_file_name = os.path.basename(source_file_name)

    rdf_output = rdf_prefixes + '\n'.join([
        rdf_template.format(

            title_uuid=data_to_uuid(title),

            title = title,

            identifier=lexeme_id,

            creator="Hassan Makki",

            publisher="Editions Geuthner",

            coverage=';\n'.join([f'dc:coverage "{escape(subj[0])}"@fr' for subj in lexeme_info.get("coverage", [])]) + ' ;' if lexeme_info.get("coverage") else "",

            subject=";\n".join([f'dc:subject "{escape(subject[0])}"@{subject[1]}' for subject in lexeme_info.get("subject", [])]) + ' ;' if lexeme_info.get("subject") else "",

            example=";\n".join([f'lex:example \"\"\"{escape(example[0])}\"\"\"@{example[1]}' for example in lexeme_info.get("example", [])]) + ' ;' if lexeme_info.get("example") else "",

            source= source_file_name,

            etymo=";\n".join([f'lex:etymo "{escape(etymo[0])}"@{etymo[1]}' for etymo in lexeme_info.get("etymo", [])])+ ' ;' if lexeme_info.get("etymo") else "",

            quic=f'dc:quic "{str(bool(lexeme_info.get("quic"))).lower()}"'+ ' ;',


            abstract=";\n".join([f'dc:abstract \"\"\"{escape(abstract[0])}\"\"\"@{abstract[1]}' for abstract in lexeme_info.get("abstract", [])]) if lexeme_info.get("abstract") else ""
        )

        for title, lexeme_data in data.items()
        for lexeme_id, lexeme_info in lexeme_data["description"].items()
    ])

    return rdf_output


def escape(s: str) -> str:

    out = ""

    for i in range(len(s)):

        match s[i]:
            case '\n':
                out += '\\n'
            case '\r':
                out += '\\r'
            case '\t' :
                out += '\\t'
            case '\f':
                out += '\\f'
            case '"' | "'":
                out += '\\' + s[i]
            case _:
                out += s[i]

    return out

##Utilisation du script
Ici, vous exécutez le script ci-dessous avec :
- En entrée : le chemin vers votre fichier docx.
- En sortie : le chemin vers votre futur fichier rdf.

In [ ]:
VERBOSE=False

def echo(s: str):
    if VERBOSE:
        print(s)

if __name__=="__main__":
    #Arguments simulés



    #--------------------CHANGEMENTS A FAIRE--------------------
    args_list=[
        "/content/kha.docx",  #--------------- ICI, CHANGER LE NOM DU FICHIER
        "--output", "/content/",  #Dossier de sortie -- ne pas changer
        "--verbose",  #Affichage détaillé -- ne pas changer
    ]
    #--------------------FIN CHANGEMENTS A FAIRE--------------------




    parser=argparse.ArgumentParser(
        prog='Extracteur Makki',
        description="Récupère les informations d'un fichier docx et en tire un fichier rdf (turtle).")

    parser.add_argument('filename', help="Chemin des fichiers docx d'entrée", nargs='+')
    parser.add_argument('--exclude', "-x", required=False, help="Chemin vers un fichier des tags à exclure (optionnel)", nargs=1, default="")
    parser.add_argument('--output', '-o', required=True, help="Dossier(s) vers lesquels exporter les données. Soit 1 seul, soit 1 par fichier d'entrée", nargs='+')
    parser.add_argument('--verbose', '-v', required=False, help="Affiche la progression du système.", action="store_true")
    parser.add_argument('--rdf', "-r", required=False, help="Stipule que les fichiers d'entrée sont déjà des fichiers json valides", action="store_true")

    # Utilisation des arguments définis par args_list
    args=parser.parse_args(args_list)

    if len(args.output) != 1:
        if len(args.output) != len(args.filename):
            raise argparse.ArgumentError("Il faut soit un seul dossier de sortie, soit un dossier par fichier d'entrée.")
        else:
            output_folder = args.output # several
    else:
        output_folder = [args.output[0]]*len(args.filename) # unique

    VERBOSE = args.verbose

    for f, d in zip(args.filename, output_folder):

        output_name = os.path.basename(f).split('.')[0]
        json_path = os.path.join(d, output_name + ".json")

        if not args.rdf:

            ed = ExtractData(f, args.exclude[0] if len(args.exclude) > 0 else "")

            echo(f"| Extraction du fichier `{f}`...")
            t0 = time.time()
            ed.extract_definition(VERBOSE)
            t1 = time.time()

            echo(f"\t> {t1-t0} s ({(t1-t0)/60} min)")

            ed.write_json_to_file(json_path)

        echo(f"\t> Export en RDF...")
        rdf = convert_to_rdf(json_path, output_name)

        with open(os.path.join(d, output_name + '.ttl'), mode="w", encoding="utf-8") as ttl:
            ttl.write(rdf)

        echo("\t> TERMINE")

    echo("TRAITEMENT TERMINE.")